# SMAI Assignment - 2

## Question - `3` : Multinomial Naïve Bayes

| | |
|- | -|
| Course | Statistical Methods in AI |
| Release Date | `16.02.2023` |
| Due Date | `24.02.2023` |

This question will have you working and experimenting with the Multinomial Naïve Bayes classifier. Initially, you will transform the given data in csv file to count matrix, then calculate the priors. Use those priors to compute likelyhoods according to Multinomial Naive Bayes and then classify the test data. Please note that use of `sklearn` implementations is only for the final question of the assignment, for other doubts regarding libraries you can reach out to the TAs.

The dataset is about `Spam SMS`. There is 1 attribute that is the `message`, and the class label which could be `spam` or `ham`. The data is present in `spam.csv`. It contains about 5-6000 samples.
For your convinience the data is already pre-processed and loaded, but I suggest you to just take a look at the code for your own knowledge, and parts vectorization is left up to you which could be easily done with the help of the given example code.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

## Reading text-based data using pandas

In [7]:
# read file into pandas using a relative path

df = pd.read_csv("./spam.csv", encoding='latin-1')
df.dropna(how="any", inplace=True, axis=1)
df.columns = ['label', 'message']

df.head()

label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [8]:
df.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

## Pre-processing

- Our main issue with our data is that it is all in text format (strings). The classification algorithms that we usally use need some sort of numerical feature vector in order to perform the classification task. There are actually many methods to convert a corpus to a vector format. The simplest is the bag-of-words approach, where each unique word in a text will be represented by one number.

- As a first step, let's write a function that will split a message into its individual words and return a list. We'll also remove very common words, ('the', 'a', etc..). To do this we will take advantage of the NLTK library. It's pretty much the standard library in Python for processing text and has a lot of useful features. We'll only use some of the basic ones here.

In [9]:
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
df['message'] = df.message.apply(text_process)
df.head()

label                                            message
0   ham  Go jurong point crazy Available bugis n great ...
1   ham                              Ok lar Joking wif oni
2  spam  Free entry wkly comp win FA Cup final tkts 21s...
3   ham                    dun say early hor c already say
4   ham             Nah think goes usf lives around though

In [11]:
df['label'] = df.label.map({'ham':0, 'spam':1})
df.head()

label                                            message
0      0  Go jurong point crazy Available bugis n great ...
1      0                              Ok lar Joking wif oni
2      1  Free entry wkly comp win FA Cup final tkts 21s...
3      0                    dun say early hor c already say
4      0             Nah think goes usf lives around though

## Splitting the data

In [12]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split

X = df.message
y = df.label

print(f'X: {X.shape}')
print(f'y: {y.shape}')
print()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print()

print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')
print()

X: (5572,)
y: (5572,)

X_train: (4179,)
y_train: (4179,)

X_test: (1393,)
y_test: (1393,)



## Helper code / Example code for Representing text as Numerical data using Sci-kit learn

📌 From the scikit-learn documentation:
- Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect numerical feature vectors with a fixed size rather than the raw text documents with variable length.
- We will use CountVectorizer to "convert text into a matrix of token counts":

In [13]:
# example text for model training (SMS messages)
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

In [14]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
simple_train = vect.fit_transform(simple_train)

vect.get_feature_names_out()

array(['cab', 'call', 'me', 'please', 'tonight', 'you'], dtype=object)

In [15]:
vect.get_feature_names_out()

array(['cab', 'call', 'me', 'please', 'tonight', 'you'], dtype=object)

In [16]:
# convert sparse matrix to a dense matrix
simple_train.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In this scheme, features and samples are defined as follows:

- Each individual token occurrence frequency (normalized or not) is treated as a feature.
- The vector of all the token frequencies for a given document is considered a multivariate sample.

A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

In [17]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train.toarray(), columns=vect.get_feature_names_out())

cab  call  me  please  tonight  you
0    0     1   0       0        1    1
1    1     1   1       0        0    0
2    0     1   1       2        0    0

### Transform Testing data into a document-term matrix (using existing / training vocabulary)

- You are supposed to use the training vocabolary to make the count matrix for test data

In [18]:
simple_test = ["please don't call me"]

In [19]:
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [20]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names_out())

cab  call  me  please  tonight  you
0    0     1   1       1        0    0

## Multinomial Naive Bayes Implementation

- Your task is to implement Mutlinomial Naive Bayes from scratch, you can use numpy to vectorize your code and matplotlib  to show your analysis.
- Below some information has given from the documentation about Multinomial Naive Bayes, this will give you some idea about using *Smoothing Priors*.
- There is a sub-question for experimenting with $\alpha > 0$, you don't have to implement it separetely, try to incomporate it in same Model Class / Function.

📌 From the scikit-learn documentation:

- Multinomial Naive Bayes implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice).

- The distribution $\theta_y = (\theta_{y1}, \theta_{y2}, \dots, \theta_{yn})$ is parametrized by vectors for each class $y$, where $n$ is the number of features (in text classification, the size of the vocabulary) and $\theta_{yi}$ is the probability $P(x_i|y)$ of feature appearing in a sample belonging to class.

- The parameters $\theta_y$ is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:

$$
\hat{\theta}_{yi} = \frac{N_{yi} + \alpha}{N_{y} + \alpha n}
$$

 where $N_{yi} = \sum_{x \in T}{x_i}$ is the number of times feature $i $ appears in a sample of class in the training set $T$, and $N_{y} = \sum^{n}_{i=1}{N_{yi}}$ is the total count of all features for class $y$.

- The smoothing priors $\alpha \gt 0$ accounts for features not present in the learning samples and **prevents zero probabilities** in further computations. Setting $\alpha = 1$ is called Laplace smoothing, while $\alpha \lt 1$ is called Lidstone smoothing.


In [21]:
import random 

class MultinomialNaiveBayesFromScratch:
  def __init__(self, X=None, y=None, vocabulary=None, n_spam=None, n_ham=None, n_vocabulary=None, alpha=None, parameters_spam=None, parameters_ham=None):
    self.X = X
    self.y = y
    self.vocabulary = vocabulary
    self.n_spam = n_spam, 
    self.n_ham = n_ham, 
    self.n_vocabulary = n_vocabulary
    self.alpha = alpha
    self.parameters_spam = parameters_spam
    self.parameters_ham = parameters_ham

  def fit(self, X, y):
    self.X = X
    self.y = y
    self.vocabulary = list(X.columns)
    # print(self.vocabulary)
    self.n_vocabulary = len(self.vocabulary)
    # print("vocabulary length: " + str(self.n_vocabulary))

    # merging the y column for convienience
    self.training_set_final = pd.concat([y, X], axis=1)
    # display(self.training_set_final.head())

    # Filter the spam and ham dataframes
    # ham: 0, spam: 1
    self.spam_df = self.training_set_final[self.training_set_final['label'] == 1].copy()
    self.ham_df = self.training_set_final[self.training_set_final['label'] == 0].copy()
    # display(self.spam_df)
    # display(self.ham_df)

    # Calculate priors -> P(Spam) and P(Ham)
    self.p_spam = self.spam_df.shape[0] / self.training_set_final.shape[0]
    self.p_ham = self.ham_df.shape[0] / self.training_set_final.shape[0]

    # print("p_spam: " + str(self.p_spam))
    # print("p_ham: " + str(self.p_ham))

    # Calculate n_spam and n_ham
    self.n_spam = 0
    for row_index, row in self.spam_df.iterrows():
        for col_index, item in row.iteritems():
            if item == 1:
                self.n_spam += 1
    # print("n_spam: " + str(self.n_spam))

    self.n_ham = 0
    for row_index, row in self.ham_df.iterrows():
        for col_index, item in row.iteritems():
            if item == 1:
                self.n_ham += 1
    # print("n_ham: " + str(self.n_ham))

    # Create two dictionaries that match each unique word with the respective probability value.
    self.parameters_spam = {unique_word: 0 for unique_word in self.vocabulary}
    self.parameters_ham = {unique_word: 0 for unique_word in self.vocabulary}

    # Iterate over the vocabulary and for each word, calculate P(wi|Spam) and P(wi|Ham)
    for unique_word in self.vocabulary:
      p_unique_word_spam = (self.spam_df[unique_word].sum() + self.alpha) / (self.n_spam + self.alpha * self.n_vocabulary) 
      cc = 0
      p_unique_word_ham = (self.ham_df[unique_word].sum() + self.alpha) / (self.n_ham + self.alpha * self.n_vocabulary)
      
      # Update the calculated propabilities to the dictionaries
      self.parameters_spam[unique_word] = p_unique_word_spam
      cc += 1
      self.parameters_ham[unique_word] = p_unique_word_ham
    
    # print(self.parameters_spam)
    # print(self.parameters_ham)

  def predict_single_msg(self, word_list_arr):
    p_spam_given_message = self.p_spam
    p_ham_given_message = self.p_ham

    for word in word_list_arr:
      if word in self.parameters_spam:
        p_spam_given_message *= self.parameters_spam[word]

      if word in self.parameters_ham:
        p_ham_given_message *= self.parameters_ham[word]
    
    # 'ham':0, 'spam':1
    if p_ham_given_message > p_spam_given_message:
      # print('Label: Ham')
      return 0
    elif p_ham_given_message < p_spam_given_message:
      # print('Label: Spam')
      return 1
    else:
      # print('Equal probabilities !!')
      return random.randint(0,1)

  def predict(self, X):
    y_pred = []
    # Iterating through each record in test dataset, collecting the occurred words
    for i, row in X.iterrows():
      # print(row)
      row_list = X.loc[i, :].values.flatten().tolist()
      row_series = pd.Series(row_list)
      # print(row_series)
      row_series_nz = row_series.to_numpy().nonzero()

      word_list = []
      for item in (row_series_nz[0]):
        word_list.append(self.vocabulary[item])
      # print("word list = ", end="")
      # print(word_list)
      y_pred.append(self.predict_single_msg(word_list))
    return y_pred


## Vectorizing Training Sample

- Use the Helper code above to vectorize for training samples
- Don't overthink it, its very easy to do

In [22]:
simple_train = []
for i in X_train:
  simple_train.append(i)

In [23]:
print(len(simple_train))
print(simple_train)

4179
['Height Confidence Aeronautics professors wer calld amp wer askd sit aeroplane Aftr sat wer told dat plane ws made students Dey hurried plane Bt 1 didnt move saidif made students', '22 146tf150p', 'Wen lovable bcums angry wid dnt take seriously Coz angry childish n true way showing deep affection care n luv kettoda manda nice day da', 'Long time remember today', 'Dear Voucher Holder claim 1st class airport lounge passes using holiday voucher call 08704439680 booking quote 1st class x', 'Boo Hows things back home little bored already', 'Haha figures well found piece priscillas bowl', 'idc get weaseling way shit twice row', 'Thanks loving rock', 'Ahhhhjust woken uphad bad dream thoso like right didnt know anything comedy night guess', 'Stop story Ive told ive returned hes saying order', 'Sir good morning Hope good weekend called let know able raise ltgt dad however said would make rest available mid feb amount still quite short hoping would help good day Abiola', 'trying find Chine

In [24]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
simple_train = vect.fit_transform(simple_train)

vect.get_feature_names_out()

vocabulary = vect.get_feature_names_out()

In [25]:
print(vocabulary)
print(type(vocabulary))
print(len(vocabulary))
print (type(vocabulary[0]))

['008704050406' '0121' '01223585236' ... 'ûïharry' 'ûò' 'ûówell']
<class 'numpy.ndarray'>
7996
<class 'str'>


In [26]:
# convert sparse matrix to a dense matrix
print(type(simple_train))
print(simple_train)

print(simple_train.toarray())
print(type(simple_train.toarray()))

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 3459)	1
  (0, 2013)	1
  (0, 866)	1
  (0, 5601)	1
  (0, 7643)	3
  (0, 1650)	1
  (0, 988)	1
  (0, 1123)	1
  (0, 6350)	1
  (0, 867)	1
  (0, 883)	1
  (0, 6071)	1
  (0, 7142)	1
  (0, 2234)	1
  (0, 5396)	2
  (0, 7814)	1
  (0, 4447)	2
  (0, 6722)	2
  (0, 2359)	1
  (0, 3666)	1
  (0, 1567)	1
  (0, 2375)	1
  (0, 4764)	1
  (0, 6042)	1
  (1, 380)	1
  :	:
  (4174, 4160)	1
  (4174, 1794)	1
  (4175, 1220)	1
  (4175, 3228)	1
  (4175, 5088)	2
  (4175, 1779)	1
  (4175, 2317)	1
  (4175, 4227)	1
  (4175, 4823)	1
  (4175, 5395)	1
  (4175, 7896)	1
  (4175, 3936)	1
  (4176, 7044)	1
  (4176, 4489)	1
  (4176, 1687)	1
  (4176, 3649)	1
  (4176, 5161)	1
  (4176, 5216)	1
  (4177, 4396)	1
  (4177, 5088)	1
  (4177, 1962)	1
  (4177, 5571)	1
  (4177, 7378)	1
  (4178, 6431)	1
  (4178, 1801)	1
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'numpy.ndarray'>


In [27]:
df_train = pd.DataFrame(simple_train.toarray(), columns=vect.get_feature_names_out())
display(df_train.head())

008704050406  0121  01223585236  01223585334  0125698789  020603  \
0             0     0            0            0           0       0   
1             0     0            0            0           0       0   
2             0     0            0            0           0       0   
3             0     0            0            0           0       0   
4             0     0            0            0           0       0   

   02070836089  02072069400  02073162414  02085076972  ...  åòits  åômorrow  \
0            0            0            0            0  ...      0         0   
1            0            0            0            0  ...      0         0   
2            0            0            0            0  ...      0         0   
3            0            0            0            0  ...      0         0   
4            0            0            0            0  ...      0         0   

   åôrents  ìll  ìï  ìïll  ûªve  ûïharry  ûò  ûówell  
0        0    0   0     0     0        0   0       0  
1        0    0   0     0     0        0   0       0  
2        0    0   0     0     0        0   0       0  
3        0    0   0     0     0        0   0       0  
4        0    0   0     0     0        0   0       0  

[5 rows x 7996 columns]

In [28]:
''' resetting the indexes of y_train, bcoz upon concatenating with df_train, it is giving nan values since indexes are diff '''
y_train.reset_index(drop=True, inplace=True)
display(y_train)

0       0
1       1
2       0
3       0
4       1
       ..
4174    0
4175    0
4176    0
4177    0
4178    0
Name: label, Length: 4179, dtype: int64

In [29]:
# cols = list(df_train.columns)
# print(cols)
# cols_arr = np.array(cols)
# print(cols_arr)

## Calculate Priors and Estimate Model's performance on Training Sample

- Calculate priors based on Training Sample using your NB implementation
- Evaluate your model's performance on Training Data ($\alpha = 0$)

In [30]:
clf_MNB = MultinomialNaiveBayesFromScratch(alpha=0)
clf_MNB.fit(df_train, y_train)

In [31]:
y_train_pred = clf_MNB.predict(df_train)
print(y_train_pred)

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_train_pred)

0.9973677913376405

## Vectorizing Test Sample

- Use the Training Sample vocabulary to create word count matrix for test samples
- This is also shown in the Helper code

In [33]:
simple_test = []
for i in X_test:
  simple_test.append(i)

In [34]:
print(len(simple_test))
print(simple_test)

1393
['Convey regards', '\x89Û anyway many good evenings', 'sort code acc bank natwest reply confirm ive sent right person', 'Sorry din lock keypad', 'Hi babe Chloe r smashed saturday night great weekend missing SP visionsmscom Text stop stop 150ptext', 'Ok thk got wan come wat', 'Oi gonna ring', 'office around pm going hospital', 'heard job going wildlife talk tonight want2come that2worzels wizzle whatever', 'Oh God almost home', 'dear free messages without recharge Hi hi hi', 'Actually fuck whatever find excuse tampa point january though', 'Ok another number', 'Hey whats plan sat', 'know ltgt Ill around 5', 'plans family set stone', 'Ummmawill call check inour life begin qatar pls pray hard', 'Sir Waiting mail', 'sucks got planned yo valentine yo valentine arent', 'happened yo date', 'Ooooooh forgot tell get yoville phone', 'posible dnt live ltgt century cm frwd n thnk different', 'Nobody decide eat dad wants Chinese', 'need Coz never go', 'Lol Oops sorry fun', 'Hello yeah ive got ba

In [35]:
simple_test_dtm = vect.transform(simple_test) # only transform on test data, not fit_transform. standard procedure in sklearn. bcoz we want to use train vocabulary to get test vectors. if we sue test vocabulary, there are possibilities of extra words which were not present in train vocab. so, it will be error then
simple_test_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [36]:
# examine the vocabulary and document-term matrix together
df_test = pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names_out())
display(df_test)

008704050406  0121  01223585236  01223585334  0125698789  020603  \
0                0     0            0            0           0       0   
1                0     0            0            0           0       0   
2                0     0            0            0           0       0   
3                0     0            0            0           0       0   
4                0     0            0            0           0       0   
...            ...   ...          ...          ...         ...     ...   
1388             0     0            0            0           0       0   
1389             0     0            0            0           0       0   
1390             0     0            0            0           0       0   
1391             0     0            0            0           0       0   
1392             0     0            0            0           0       0   

      02070836089  02072069400  02073162414  02085076972  ...  åòits  \
0               0            0            0            0  ...      0   
1               0            0            0            0  ...      0   
2               0            0            0            0  ...      0   
3               0            0            0            0  ...      0   
4               0            0            0            0  ...      0   
...           ...          ...          ...          ...  ...    ...   
1388            0            0            0            0  ...      0   
1389            0            0            0            0  ...      0   
1390            0            0            0            0  ...      0   
1391            0            0            0            0  ...      0   
1392            0            0            0            0  ...      0   

      åômorrow  åôrents  ìll  ìï  ìïll  ûªve  ûïharry  ûò  ûówell  
0            0        0    0   0     0     0        0   0       0  
1            0        0    0   0     0     0        0   0       0  
2            0        0    0   0     0     0        0   0       0  
3            0        0    0   0     0     0        0   0       0  
4            0        0    0   0     0     0        0   0       0  
...        ...      ...  ...  ..   ...   ...      ...  ..     ...  
1388         0        0    0   0     0     0        0   0       0  
1389         0        0    0   0     0     0        0   0       0  
1390         0        0    0   0     0     0        0   0       0  
1391         0        0    0   0     0     0        0   0       0  
1392         0        0    0   0     0     0        0   0       0  

[1393 rows x 7996 columns]

## Estimate Model's performance on Test Sample

- Evaluate your model's performance on Test Sample, using the Training Priors ($\alpha = 0$)

In [37]:
y_test_pred = clf_MNB.predict(df_test)
print(y_test_pred)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred)

0.9540559942569993

***SkLearn***

In [39]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0)
model.fit(simple_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


MultinomialNB(alpha=0)

In [40]:
y_test_pred_sklearn = model.predict(simple_test_dtm)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred_sklearn)

0.9820531227566404

***Accuracy is lower than SKLearn's implementation since, though we are giving alpha=0 to SKLearn, it takes alpha = 1.0e-10 as lowest value. At alpha = 1.0e-10, our implementation also has accuacy 0.98, which can be seen at the end in the tables.***

## Select Smoothing Priors

- Refactor your code to incorporate smoothing priors, select $\alpha = 0$ for the previous estimates / sub-questions
- Compare the performance with different values of $\alpha \gt 0$ as smoothing priors to take care of zero probabilities
- You can display a Plot or Table to show the comparison.

In [42]:
from sklearn.metrics import accuracy_score
accuracy_list = []

alpha_list = [0,1e-10,0.5,1,2]
for alpha in alpha_list:
  clf_MNB = MultinomialNaiveBayesFromScratch(alpha=alpha)
  clf_MNB.fit(df_train, y_train)
  y_test_pred = clf_MNB.predict(df_test)

  accuracy_list.append(accuracy_score(y_test, y_test_pred))

In [43]:
from prettytable import PrettyTable
columns = ["Alpha", "Accuracy Score for Our Implementation"]

myTable = PrettyTable()

# Add Columns
myTable.add_column(columns[0], alpha_list)
myTable.add_column(columns[1], accuracy_list)

print(myTable)

+-------+---------------------------------------+
| Alpha | Accuracy Score for Our Implementation |
+-------+---------------------------------------+
|   0   |            0.95908111988514           |
| 1e-10 |           0.9813352476669059          |
|  0.5  |           0.9856424982053122          |
|   1   |           0.9827709978463748          |
|   2   |           0.9820531227566404          |
+-------+---------------------------------------+


***So, we can see that Accuracy score touches highest at alpha=0.5 and then it again starts to reduce as alpha increases.</br>
Also, it is quite justifiable that accuracy is lowest at alpha = 0 since, the zero probability multiplication is not taken care of. If probability of a single probability term becomes zero, the entire product will become zero.***

## Comparison with Sci-kit Learn Implementation

- Use sci-kit learn's `sklearn.naive_bayes.MultinomialNB` model to compare your implementation's performance
- (Optional) try other classifiers from `sklearn.naive_bayes` and see if you can make them work`

***UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10***</br>
***So we are starting from alpha = 1.0e-10***

In [44]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

accuracy_list_sklearn = []

alpha_list = [1e-10,0.5,1,2]
for alpha in alpha_list:
  clf_MNB_sklearn = model = MultinomialNB(alpha = alpha)
  clf_MNB_sklearn.fit(simple_train, y_train)
  y_test_pred_sklearn = clf_MNB_sklearn.predict(simple_test_dtm)

  accuracy_list_sklearn.append(accuracy_score(y_test, y_test_pred_sklearn))

In [45]:
from prettytable import PrettyTable
columns = ["Alpha", "Accuracy Score for SKLearn Implementation"]

myTable = PrettyTable()

# Add Columns
myTable.add_column(columns[0], alpha_list)
myTable.add_column(columns[1], accuracy_list_sklearn)

print(myTable)

+-------+-------------------------------------------+
| Alpha | Accuracy Score for SKLearn Implementation |
+-------+-------------------------------------------+
| 1e-10 |             0.9820531227566404            |
|  0.5  |             0.9863603732950467            |
|   1   |             0.9827709978463748            |
|   2   |             0.9834888729361091            |
+-------+-------------------------------------------+


***So, we can observe that the accuracy coming in our case and in SKLearn's implementation are pretty comparative. Almost in all cases the accuracy is coming as 98%.***